In [1]:
import json
import os
import string
import re
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
import lda


In [2]:
#setting a path
os.chdir('/Users/dorsasafaei/Documents/computational-social-media/assignment#3')

In [3]:
# 1) reading json file
tweetFile = open('./data/covidtrack_50K.json', 'r') 
tweetLines = tweetFile.readlines() 
data = []
for tweet in tweetLines:
    data.append(json.loads(tweet, strict=False))

In [4]:
lengthOfData = len(data)

In [5]:
# join tweets
text = list(map(lambda x: x['text'], data))

In [6]:
# 2.a)lower case text
text = list(map(lambda x: x.lower(), text))

In [15]:
# 2.b)remove non-alphanumeric we did not consider _ - as alphanumeric
text = list(map(lambda x: re.sub(r'[^a-z0-9 ]+', " ", x), text))

In [16]:
# 2.c)
for count in range(len(text)):  
    # split text   
    text_split = text[count].split(" ")
    # remove words with less than 3 char
    text[count] = " ".join(list(filter(lambda x: len(x)>3, text_split)))

In [17]:
# 2.e) converting a word to its base form
wnl = WordNetLemmatizer()
for count in range(len(text)): 
        text[count] = " ".join(list(map(lambda x: wnl.lemmatize(x), text[count].split(" ") ) ) )

In [18]:
# 2.f) remove stop words
stop_words =  set(stopwords.words('english'))
for count in range(len(text)): 
    text[count] = " ".join(list(filter(lambda x: x not in stop_words, word_tokenize(text[count]))))

In [19]:
# create unique list of words per document
wordUniq = ""
for count in range(len(text)): 
    wordUniq = wordUniq + " " + " ".join(set(word_tokenize(text[count])))

In [28]:
# 3.a) Remove words that occur in less than 10 documents, and words that occur in more than 90% of the documents
# however it is more efficient to keep the list of words to keep as it is a smaller set
wordUniqTokenize = word_tokenize(wordUniq)
words2Remove = []
for x in set(wordUniqTokenize):
    if wordUniqTokenize.count(x) < 11:
        words2Remove.append(x)
    if wordUniqTokenize.count(x) >= round(.9 * len(text)):
        words2Remove.append(x)

for count in range(len(text)): 
    text[count] = " ".join(list(filter(lambda x: x not in words2Remove, word_tokenize(text[count]))))

In [20]:
# 3.b) Build a dictionary
wordUniqTokenize = word_tokenize(wordUniq)
dictionary = []
for x in set(wordUniqTokenize):
    if (wordUniqTokenize.count(x) > 10) and (wordUniqTokenize.count(x) < round(.9 * len(text))):
        dictionary.append(x)

In [33]:
# number of words to keep,as in the bound
len(dictionary)

4404

In [34]:
# number of words to remove because not in the threshold
len(words2Remove)

38290

In [37]:
# check if the two calculation method for words in and out make to the whole number
len(words2Remove) + len(dictionary) == len(set(wordUniqTokenize))

True

In [22]:
# keep the words after filter unnecassry words for topic extraction
for count in range(len(text)): 
    text[count] = " ".join(list(filter(lambda x: x  in dictionary, word_tokenize(text[count]))))

In [23]:
# 3.c: extract word freq per document
word_freq = np.zeros([len(text), len(dictionary)], dtype=int)
for count in range(len(text)): 
    text_tokenize = word_tokenize(text[count])
    word_freq[count,] = list(map(lambda x: text_tokenize.count(x), dictionary))


In [24]:
# check any row is zero (there should not be any)
sum(np.sum(word_freq, 0) == 0)

0

In [25]:
# 3.d) Apply the LDA model (n_iter=1500, random_state=1)
model = lda.LDA(n_topics=20, n_iter=1500, random_state=1)
model.fit(word_freq)

INFO:lda:n_documents: 50000
INFO:lda:vocab_size: 4404
INFO:lda:n_words: 415699
INFO:lda:n_topics: 20
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -4846181
INFO:lda:<10> log likelihood: -2992754
INFO:lda:<20> log likelihood: -2850711
INFO:lda:<30> log likelihood: -2821051
INFO:lda:<40> log likelihood: -2810245
INFO:lda:<50> log likelihood: -2803025
INFO:lda:<60> log likelihood: -2799259
INFO:lda:<70> log likelihood: -2796197
INFO:lda:<80> log likelihood: -2793214
INFO:lda:<90> log likelihood: -2791447
INFO:lda:<100> log likelihood: -2788975
INFO:lda:<110> log likelihood: -2789031
INFO:lda:<120> log likelihood: -2787932
INFO:lda:<130> log likelihood: -2787089
INFO:lda:<140> log likelihood: -2785746
INFO:lda:<150> log likelihood: -2785018
INFO:lda:<160> log likelihood: -2783960
INFO:lda:<170> log likelihood: -2780791
INFO:lda:<180> log likelihood: -2781155
INFO:lda:<190> log likelihood: -2779919
INFO:lda:<200> log likelihood: -2778245
INFO:lda:<210> log likelihood: -2777076
INFO:lda

In [27]:
# 3.e
topicsModel = model.topic_word_
topicTopWords = 13
for i, topicDist in enumerate(topicsModel):
    topics= np.array(dictionary)[np.argsort(topicDist)][:-topicTopWords:-1]
    print('Topic {}: {}'.format(i, ' '.join(topics)))


Topic 0: death breaking spike past toll 24hrs piersmorgan number total fatality recorded horribly
Topic 1: http covid19 briefing medium drtedros live conference iran press watch covid pandemic
Topic 2: trump realdonaldtrump china country warned death navarro people threat peter would tomfitton
Topic 3: http boris johnson minister care prime know intensive borisjohnson emergency state japan
Topic 4: death south korea http rate trump world people first state crisis davidcorndc
Topic 5: http stay people medical nurse ajenglish staff home tackle continues indoors urging
Topic 6: people crisis current continues suffering accelerate drastically occupant teampelosi iran maga regime
Topic 7: case death http total confirmed covid19 number covid report positive reported update
Topic 8: http covid19 covid china mask corona virus pandemic read covid2019 head stayhome
Topic 9: pandemic play point feeling realcandaceo watching arrived exact called patient quicktake help
Topic 10: http lockdown covid

In [192]:
# no word filtering: Transform each document to a vectorized form by computing the frequency of each word. 
list(map(lambda x: {x:wordUniqTokenize.count(x)}, set(wordUniqTokenize))) 

'sheltercymru': 1},
 {'httpstcoddukwpx0np': 1},
 {'httpstco2izddprs0w': 1},
 {'depraved': 1},
 {'httpstcod0kcfdncx9': 1},
 {'wahleyworldchanhobmakanju': 1},
 {'iamstayinghome': 1},
 {'sxdoc': 38},
 {'httpstcoxtbxfj4rga': 1},
 {'louder': 3},
 {'httpstcocwgpk46qvg': 1},
 {'httpstcosyjecsiju3': 1},
 {'mentio': 3},
 {'ruptly': 71},
 {'evidencethe': 1},
 {'hourhttpstcoyu7t74nkms': 1},
 {'poured': 3},
 {'arogyasetu': 51},
 {'meghanmccain': 1},
 {'whelp': 2},
 {'httpstcoxi2qayoitw': 2},
 {'httpstcoypkm2wdpiv': 1},
 {'inhumane': 31},
 {'launchin': 1},
 {'connect_ie': 1},
 {'drive-in': 2},
 {'2-12': 1},
 {'morningfix': 1},
 {'congressman': 12},
 {'httpstcocjuwnxoc3j': 1},
 {'pune': 18},
 {'senegal': 36},
 {'nakedleader': 1},
 {'neoavatara': 1},
 {'generated': 29},
 {'ladanski_1': 1},
 {'netherlandscartooningforsolidaritycoronavirusca': 1},
 {'httpstcommnn3pkghr': 1},
 {'cheryl': 1},
 {'moloch': 2},
 {'httpstcolvegujycer': 1},
 {'lhmandetta': 1},
 {'short-medium': 1},
 {'sandyryanpopp': 1},
 {'h